In [3]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly
import cufflinks as cf
cf.go_offline()
import warnings 
warnings.filterwarnings('ignore')


from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [4]:
data=pd.read_csv('disease_prediction\Training.csv')

In [5]:
data.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [22]:
columns=list(data.columns.values)

diseases=list(data['prognosis'].unique())
mapping={}
for i,dis in enumerate(diseases):
    mapping[dis]=i
    
invmapping={v:k for k,v in mapping.items()}

data['prognosis']=data['prognosis'].map(mapping)

In [23]:
X=np.array(data.iloc[:,:-1])

In [24]:
y=np.array(data.iloc[:,-1])[:,np.newaxis]

In [25]:
m=len(y)
k=41
ones=np.ones([m,1])
X=np.hstack([ones,X])

In [26]:
m,n=X.shape
print(X.shape,y.shape)

(4920, 133) (4920, 1)


In [10]:
theta=np.zeros([k,n])

In [11]:
print(theta.shape,X.shape,y.shape)

(41, 133) (4920, 133) (4920, 1)


In [13]:
def sigmoid(X):
    return 1/1+np.exp(-X)

In [14]:
def costfunction(theta,X,y,lmbda):
    temp=np.dot(y.T,np.log(sigmoid(np.dot(X,theta))))+np.dot((1-y).T,np.log(1-sigmoid(np.dot(X,theta))))
    return (temp/(-m))+np.sum(theta[1:]**2)*((lmbda)/(2*m))

In [15]:
def gradient(theta,X,y,lmbda):
    temp=sigmoid(np.dot(X,theta))-y
    temp=(np.dot(X.T,temp)/m)+theta * lmbda / m
    temp[0]=temp[0]-theta[0] * lmbda / m
    return temp

In [42]:
import scipy.optimize as opt
lmbda =1001
k=41
theta=np.zeros((k,n))

for i in range(k):
    digit_class=i 
    theta[i]=opt.fmin_cg(f=costfunction,x0=theta[i],fprime=gradient,args=(X,(y==digit_class).flatten(),lmbda),maxiter=50)

Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluations: 112
Optimization terminated successfully.
         Current function value: nan
         Iterations: 1
         Function evaluations: 112
         Gradient evaluatio

In [39]:
theta=np.ones((k,n))
costfunction(theta[0],X,y==0,100000000000000000)

array([nan])

In [40]:
#the cost function is returning nan

array([258.44337806])

In [70]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [110]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [111]:
oneall=OneVsRestClassifier(LinearSVC(random_state=0,multi_class="ovr")).fit(X_train, y_train)

In [112]:
oneall.predict(X_test)

array([29, 19, 25, ..., 19,  8,  1], dtype=int64)

In [114]:
oneall.score(X_test,y_test)

1.0